In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

In [4]:
volume_dir = '/home/yuncong/csd395/CSHL_volumes/'

labels_twoSides = []
labels_twoSides_indices = {}
with open(volume_dir+'/MD589/volume_MD589_annotation_withOuterContour_labelIndices.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        name, index = line.split()
        labels_twoSides.append(name)
        labels_twoSides_indices[name] = int(index)

In [6]:
labelMap_sidedToUnsided = {name: name if '_' not in name else name[:-2] for name in labels_twoSides_indices.keys()}
labels_unsided = ['BackG'] + sorted(set(labelMap_sidedToUnsided.values()) - {'BackG', 'outerContour'}) + ['outerContour']

In [8]:
atlasProjected_volume = bp.unpack_ndarray_file(os.path.join(volume_dir, 'MD589/volume_MD589_annotation_withOuterContour.bp'))
atlas_ydim, atlas_xdim, atlas_zdim = atlasProjected_volume.shape
print atlas_xdim, atlas_ydim, atlas_zdim

838 459 534


In [6]:
def parallel_where(l):
    w = np.where(atlasProjected_volume == l)
    return np.c_[w[1], w[0], w[2]]

t = time.time()

atlas_nzs = Parallel(n_jobs=16)(delayed(parallel_where)(labels_twoSides_indices[name]) 
                                for name in labels_twoSides[1:])

atlas_nzs = dict(zip(labels_twoSides[1:], atlas_nzs))

print time.time() - t, 'seconds'

7.11578392982 seconds


In [ ]:
atlas_xmin, atlas_ymin, atlas_zmin = np.min([np.min(nzs, axis=1) for nzs in atlas_nzs.itervalues()], axis=0)
atlas_xmax, atlas_ymax, atlas_zmax = np.max([np.max(nzs, axis=1) for nzs in atlas_nzs.itervalues()], axis=0)
print atlas_xmin, atlas_xmax, atlas_ymin, atlas_ymax, atlas_zmin, atlas_zmax

atlas_centroid = np.array([.5*atlas_xmin+.5*atlas_xmax, .5*atlas_ymin+.5*atlas_ymax, .5*atlas_zmin+.5*atlas_zmax])
print atlas_centroid

atlas_cx, atlas_cy, atlas_cz = atlas_centroid

In [ ]:
downsample_factor = 16

section_thickness = 20 # in um
xy_pixel_distance_lossless = 0.46
xy_pixel_distance_tb = xy_pixel_distance_lossless * 32 # in um, thumbnail
# factor = section_thickness/xy_pixel_distance_lossless

xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
z_xy_ratio_downsampled = section_thickness / xy_pixel_distance_downsampled

In [ ]:
atlasAlignOptLogs_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_atlasAlignOptLogs'
if not os.path.exists(atlasAlignOptLogs_dir):
    os.makedirs(atlasAlignOptLogs_dir)

In [ ]:
atlasAlignParams_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_atlasAlignParams'
if not os.path.exists(atlasAlignParams_dir):
    os.makedirs(atlasAlignParams_dir)

In [ ]:
annotationsViz_rootdir = '/oasis/projects/nsf/csd395/yuncong/CSHL_annotaionsPojectedViz'

In [ ]:
stack = 'MD594'

In [ ]:
with open(atlasAlignParams_dir + '/%(stack)s/%(stack)s_3dAlignParams.txt' % {'stack': stack}, 'r') as f:
    lines = f.readlines()
T_final = map(float, lines[1].strip().split())

In [ ]:
atlas_pts_centered = {}
for l in labels_twoSides[1:]:
    atlas_pts_centered[l] = atlas_nzs[l] - atlas_centroid

In [ ]:
section_bs_begin, section_bs_end = section_range_lookup[stack]
print section_bs_begin, section_bs_end

# (volume_xmin, volume_xmax, volume_ymin, volume_ymax, volume_zmin, volume_zmax) = \
# np.loadtxt(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_limits.txt' % {'stack': stack}), dtype=np.int)

# map_z_to_section = {}
# for s in range(section_bs_begin, section_bs_end+1):
#     for z in range(int(z_xy_ratio_downsampled*s) - volume_zmin, int(z_xy_ratio_downsampled*(s+1)) - volume_zmin + 1):
#         map_z_to_section[z] = s

# global volume2_allLabels
# volume2_allLabels = {}

# for l in labels_unsided[1:]:

#     print l

#     volume2 = bp.unpack_ndarray_file(os.path.join(volume_dir, 'volume_%(stack)s_scoreMap_%(label)s.bp' % \
#                                                   {'stack': stack, 'label': l}))

#     volume2_cropped = volume2[volume_ymin:volume_ymax+1, volume_xmin:volume_xmax+1]
#     # copy is important, because then you can delete the large array

#     volume2_allLabels[l] = volume2_cropped.copy()

#     del volume2, volume2_cropped

test_ydim, test_xdim, test_zdim = volume2_allLabels.values()[0].shape
test_centroid = (.5*test_xdim, .5*test_ydim, .5*test_ydim)

print test_xdim, test_ydim, test_zdim
print test_centroid

In [ ]:
d2Sdxyz2 = {}

for l in labels[1:-1]:
    
    print l
    
    t = time.time()
    
    gxx = load_hdf(volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gxx.hdf' % {'stack': stack, 'lab': l}).astype(np.float16)
    gxy = load_hdf(volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gxy.hdf' % {'stack': stack, 'lab': l}).astype(np.float16)
    gxz = load_hdf(volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gxz.hdf' % {'stack': stack, 'lab': l}).astype(np.float16)
    gyx = load_hdf(volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gyx.hdf' % {'stack': stack, 'lab': l}).astype(np.float16)
    gyy = load_hdf(volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gyy.hdf' % {'stack': stack, 'lab': l}).astype(np.float16)
    gyz = load_hdf(volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gyz.hdf' % {'stack': stack, 'lab': l}).astype(np.float16)
    gzx = load_hdf(volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gzx.hdf' % {'stack': stack, 'lab': l}).astype(np.float16)
    gzy = load_hdf(volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gzy.hdf' % {'stack': stack, 'lab': l}).astype(np.float16)
    gzz = load_hdf(volume_dir + '/volume_%(stack)s_scoreMap_%(lab)s_gzz.hdf' % {'stack': stack, 'lab': l}).astype(np.float16)
    
    print time.time() - t
    
    d2Sdxyz2[l] = [gxx, gxy, gxz, gyx, gyy, gyz, gzx, gzy, gzz]

In [ ]:
def compute_score_hessian(T):
    
#     Tm = np.reshape(T, (3,4))
#     tx, ty, tz = Tm[:, 3]
#     A = Tm[:, :3]

    d2MdT2 = np.zeros((12, 12))
    
#     for l in range(1, n_labels):
    for l in labels_twoSides[1:-1]:
        
        name_unsided = labelMap_sidedToUnsided[l]
        
        pts_prime = transform_points(T, pts_centered=atlas_pts_centered[l], c_prime=test_centroid).astype(np.int)
            
        xs_prime, ys_prime, zs_prime = pts_prime.T

        valid = (xs_prime >= 0) & (ys_prime >= 0) & (zs_prime >= 0) & \
            (xs_prime < test_xdim) & (ys_prime < test_ydim) & (zs_prime < test_zdim)

        if np.count_nonzero(valid) > 0:

            xs_prime_valid = xs_prime[valid]
            ys_prime_valid = ys_prime[valid]
            zs_prime_valid = zs_prime[valid]
            
            dxs, dys, dzs = atlas_pts_centered[l][valid].T

            Sxx_full, Sxy_full, Sxz_full, Syx_full, Syy_full, Syz_full, Szx_full, Szy_full, Szz_full = d2Sdxyz2[name_unsided]
            Sxx = Sxx_full[ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Sxy = Sxy_full[ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Sxz = Sxz_full[ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Syx = Syx_full[ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Syy = Syy_full[ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Syz = Syz_full[ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Szx = Szx_full[ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Szy = Szy_full[ys_prime_valid, xs_prime_valid, zs_prime_valid]
            Szz = Szz_full[ys_prime_valid, xs_prime_valid, zs_prime_valid]

            rx = np.c_[Sxx*dxs, Sxx*dys, Sxx*dzs, Sxx, Sxy*dxs, Sxy*dys, Sxy*dzs, Sxy, Sxz*dxs, Sxz*dys, Sxz*dzs, Sxz]
            ry = np.c_[Syx*dxs, Syx*dys, Syx*dzs, Syx, Syy*dxs, Syy*dys, Syy*dzs, Syy, Syz*dxs, Syz*dys, Syz*dzs, Syz]
            rz = np.c_[Szx*dxs, Szx*dys, Szx*dzs, Szx, Szy*dxs, Szy*dys, Szy*dzs, Szy, Szz*dxs, Szz*dys, Szz*dzs, Szz]
            r1 = (rx*dxs[:,None]).sum(axis=0)
            r2 = (rx*dys[:,None]).sum(axis=0)
            r3 = (rx*dzs[:,None]).sum(axis=0)
            r4 = rx.sum(axis=0)
            r5 = (ry*dxs[:,None]).sum(axis=0)
            r6 = (ry*dys[:,None]).sum(axis=0)
            r7 = (ry*dzs[:,None]).sum(axis=0)
            r8 = ry.sum(axis=0)
            r9 = (rz*dxs[:,None]).sum(axis=0)
            r10 = (rz*dys[:,None]).sum(axis=0)
            r11 = (rz*dzs[:,None]).sum(axis=0)
            r12 = rz.sum(axis=0)
            
            d2MdT2_l = np.vstack([r1, r2, r3, r4, r5, r6, r7, r8, r9, r10, r11, r12])
        
            del xs_prime_valid, ys_prime_valid, zs_prime_valid
            del dxs, dys, dzs
            del Sxx_full, Sxy_full, Sxz_full, Syx_full, Syy_full, Syz_full, Szx_full, Szy_full, Szz_full
            del Sxx, Sxy, Sxz, Syx, Syy, Syz, Szx, Szy, Szz
            del rx, ry, rz
            del r1, r2, r3, r4, r5, r6, r7, r8, r9, r10, r11, r12
            
        
            d2MdT2 += d2MdT2_l
        
            del d2MdT2_l
        del xs_prime, ys_prime, zs_prime, valid
    
    return d2MdT2

In [ ]:
H = compute_score_hessian(T_final)
plt.stem(range(12), H.diagonal());
print 'mean curvature:', np.sum(H.diagonal())
print 'mean curvature related to translations:', np.sum(H.diagonal()[[3,7,11]])

In [ ]:
plt.matshow(H, cmap=plt.cm.RdBu, vmin=-1e5, vmax=1e5);
plt.colorbar();

In [ ]:
# check hessians at neighborhoods of the optimum
for _ in range(10):
    Hr = compute_score_hessian(np.array(T_final) + [0,0,0,np.random.randint(-10,10),
                                                   0,0,0,np.random.randint(-10,10),
                                                   0,0,0,np.random.randint(-10,10)])
    print 'mean curvature:', np.sum(Hr.diagonal())
    print 'mean curvature related to translations:', np.sum(Hr.diagonal()[[3,7,11]])